In [1]:
# Makes report plot
# improt and define basic params
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
from pathlib import Path
from os import listdir, path, makedirs
sns.set(style="darkgrid")
%matplotlib inline

mpl.rcParams["savefig.dpi"] = 200
# Calibrated data dir
calibrated_base_dir = Path("..")/"../new_calibrate_full/"
uncalibrated_base_dir = Path("..")/"../uncalibrated_data/"
calibration_curve_base_dir = Path("..")/"../calibration_curve/"
calibrated_table_dir = Path("..")/"../tables/"
uncalibrated_table_dir = Path("..")/"../uncalibrated_tables/"

# Class to use
import sys
sys.path.append("../../python")
from auditing_setup.election_setting import Election

report_dir = Path('/Users/Dovermore/Documents/Research/AustralianElectionAuditing/ReportRelated/school_report/')
report_figure_dir = report_dir/'figures'
report_tabular_dir = report_dir/'tabular'

In [5]:
legend_order = ['bravo    | p=0.7 ',
                'bravo    | p=0.55 ',
                'bravo    | p=0.52 ',
                'bravo_without_replacement | p=0.7 ',
                'bravo_without_replacement | p=0.55 ',
                'bravo_without_replacement | p=0.52 ',
                'bayesian | a=1 | b=1 ',
                'bayesian_with_replacement | a=1 | b=1 ',
                'truncated_bayesian | a=1 | b=1 ',
                'clip     ', 
                'max_sprt ']

def get_position(name):
    for i, legend_pattern in enumerate(legend_order):
        if name.lower().startswith(legend_pattern):
            return i
    else:
        assert False

def reorder_df(df):
    return df.loc[sorted(df.index, key=get_position)]

mpl.rcParams["figure.figsize"] = [8, 6]

def read_csv(file_path, reorder=True):
    df = pd.read_csv(file_path)
    cols = []
    for i in df.columns:
        try:
            cols.append(float(i))
        except Exception as e:
            cols.append(i)
    df.columns = cols
    df.set_index('legend', inplace=True)
    if reorder:
        df = reorder_df(df)
    return df.reset_index()

def get_num_cols(df):
    cols = []
    for i in df.columns:
        try:
            float(i)
            cols.append(float(i))
        except Exception as e:
            pass
    return cols

def format_legend(ax=None, legend=None, how=0):
    if ax is not None:
        legend_ = ax.legend_
    elif legend is not None:
        legend_ = legend
    else:
        return
    
    for text in legend_.texts:
        if how == 0:
            raw = text.get_text()
            text.set_text(raw.replace(" | ", ", ").replace(" ", "").rsplit(",", 1)[0])
        elif how == 1:
            raw = text.get_text()
            print(raw.rsplit(",", 1)[0].replace("_", " "))
            text.set_text(raw.rsplit(",", 1)[0].replace("_", " "))
    return legend_

def make_legend(df, ax, outside=False, *args, **kwargs):
    if outside:
        ax.legend(labels=[basename+" "+parameter for i, (basename, parameter) in 
                          df[['basename_with_type', 'parameter']].drop_duplicates()
                          .iterrows()],
                 bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.,
                 *args, **kwargs)
    else:
        ax.legend(labels=[basename+" "+parameter for i, (basename, parameter) in 
                          df[['basename_with_type', 'parameter']].drop_duplicates()
                          .iterrows()], *args, **kwargs)

In [59]:
def pivot_widetable(df, names, groupers, *args, **kwargs):
    df_long = df.melt(*args, **kwargs)
    for name, grouper in zip(names, groupers):
        group = [grouper(i) for i in df_long['variable']]
        df_long[name] = group
    return df_long

names = [
    'type'
    'basename'
    'basename_with_type'
    'parameter'
    'parameter_with_type'
]

basename_map = {'bayesian ': 'Bayesian',
                'max_sprt ': 'MaxBRAVO',
                'bravo    ': 'BRAVO',
                'clip     ': 'Clip',
                'bravo_without_replacement ': 'BRAVO',
                'truncated_bayesian ': 'MartInf',
                'bayesian_with_replacement ': 'Bayesian'}

type_map = {'bayesian ': 'BetaBinomial',
            'max_sprt ': '',
            'bravo    ': 'Binomial',
            'clip     ': '',
            'bravo_without_replacement ': 'Hypergeometric',
            'truncated_bayesian ': '',
            'bayesian_with_replacement ': 'Beta'}

basename_map_with_type = {'bayesian ': 'Bayesian (BetaBinomial)',
                          'max_sprt ': 'MaxBRAVO',
                          'bravo    ': 'BRAVO (Binomial)',
                          'clip     ': 'Clip',
                          'bravo_without_replacement ': 'BRAVO (Hypergeometric)',
                          'truncated_bayesian ': 'MartInf',
                          'bayesian_with_replacement ': 'Bayesian (Beta)'}

def group_basename(name):
    name = name.split('|')[0]
    return basename_map[name]

def group_type(name):
    name = name.split('|')[0]
    return type_map[name]

def group_basename_with_type(name):
    name = name.split('|')[0]
    return basename_map_with_type[name]

def group_parameter(name):
    _, name = name.split('|', 1)
    name = name.rsplit('|', 1)[0] if len(name.rsplit('|', 1)) > 1 else ''
    name = name.replace('|', ',')
    name = name.replace(' ', '')
    return name

def group_parameter_with_type(name):
    _, name = name.split('|', 1)
    type = group_type(_)
    name = name.rsplit('|', 1)[0] if len(name.rsplit('|', 1)) > 1 else ''
    name = name.replace('|', ',')
    name = name.replace(' ', '')
    return type+' '+name
    
groupers = [
    group_type,
    group_basename,
    group_basename_with_type,
    group_parameter,
    group_parameter_with_type
]

def prep_legend(df):
    parameters = list(df.legend.apply(group_parameter))
    basenames_with_type = list(df.legend.apply(group_basename_with_type))
    df['legend'] = [name + ' ' + param for name, param in zip(basenames_with_type, parameters)]
    df.set_index('legend', inplace=True)
    return df

In [82]:
# 1. Basic 500/500 plot without replacement

# data
election = Election(500, 500, 0.5, 1, False)

data_type = 'power.csv'
data_path = calibrated_table_dir/str(election)/data_type
data = prep_legend(read_csv(data_path))
filter_cols = filter(lambda x:x>=0.5, data.columns)
filter_cols = [col for i,col in enumerate(filter_cols) if i % 2 == 0]
data.index.name = 'Audit Method'
data[filter_cols].round(2).to_latex(report_tabular_dir/'power{}_tabular.tex'.format(election.n))

data_type = 'unconditional_mean.csv'
data_path = calibrated_table_dir/str(election)/data_type
data = prep_legend(read_csv(data_path))
filter_cols = filter(lambda x:x>=0.5, data.columns)
filter_cols = [col for i,col in enumerate(filter_cols) if i % 2 == 0]
data.index.name = 'Audit Method'
data[filter_cols].round(2).to_latex(report_tabular_dir/'unconditional_mean{}_tabular.tex'.format(election.n))

data_type = 'unconditional_mean_with_recount.csv'
data_path = calibrated_table_dir/str(election)/data_type
data = prep_legend(read_csv(data_path))
filter_cols = filter(lambda x:x>=0.5, data.columns)
filter_cols = [col for i,col in enumerate(filter_cols) if i % 2 == 0]
data.index.name = 'Audit Method'
data[filter_cols].round(2).to_latex(report_tabular_dir/'unconditional_mean_with_recount{}_tabular.tex'.format(election.n))

In [83]:
# 1. Basic 5000/500 plot without replacement

# data
election = Election(5000, 500, 0.5, 1, False)

data_type = 'power.csv'
data_path = calibrated_table_dir/str(election)/data_type
data = prep_legend(read_csv(data_path))
filter_cols = filter(lambda x:x>=0.5, data.columns)
filter_cols = [col for i,col in enumerate(filter_cols) if i % 2 == 0]
data.index.name = 'Audit Method'
data[filter_cols].round(2).to_latex(report_tabular_dir/'power{}_tabular.tex'.format(election.n))

data_type = 'unconditional_mean.csv'
data_path = calibrated_table_dir/str(election)/data_type
data = prep_legend(read_csv(data_path))
filter_cols = filter(lambda x:x>=0.5, data.columns)
filter_cols = [col for i,col in enumerate(filter_cols) if i % 2 == 0]
data.index.name = 'Audit Method'
data[filter_cols].round(2).to_latex(report_tabular_dir/'unconditional_mean{}_tabular.tex'.format(election.n))

data_type = 'unconditional_mean_with_recount.csv'
data_path = calibrated_table_dir/str(election)/data_type
data = prep_legend(read_csv(data_path))
filter_cols = filter(lambda x:x>=0.5, data.columns)
filter_cols = [col for i,col in enumerate(filter_cols) if i % 2 == 0]
data.index.name = 'Audit Method'
data[filter_cols].round(2).to_latex(report_tabular_dir/'unconditional_mean_with_recount{}_tabular.tex'.format(election.n))

In [84]:
# 1. Basic 10000/1000 plot without replacement

# data
election = Election(10000, 1000, 0.5, 1, False)

data_type = 'power.csv'
data_path = calibrated_table_dir/str(election)/data_type
data = prep_legend(read_csv(data_path))
filter_cols = filter(lambda x:x>=0.5, data.columns)
filter_cols = [col for i,col in enumerate(filter_cols) if i % 2 == 0]
data.index.name = 'Audit Method'
data[filter_cols].round(2).to_latex(report_tabular_dir/'power{}_tabular.tex'.format(election.n))

data_type = 'unconditional_mean.csv'
data_path = calibrated_table_dir/str(election)/data_type
data = prep_legend(read_csv(data_path))
filter_cols = filter(lambda x:x>=0.5, data.columns)
filter_cols = [col for i,col in enumerate(filter_cols) if i % 2 == 0]
data.index.name = 'Audit Method'
data[filter_cols].round(2).to_latex(report_tabular_dir/'unconditional_mean{}_tabular.tex'.format(election.n))

data_type = 'unconditional_mean_with_recount.csv'
data_path = calibrated_table_dir/str(election)/data_type
data = prep_legend(read_csv(data_path))
filter_cols = filter(lambda x:x>=0.5, data.columns)
filter_cols = [col for i,col in enumerate(filter_cols) if i % 2 == 0]
data.index.name = 'Audit Method'
data[filter_cols].round(2).to_latex(report_tabular_dir/'unconditional_mean_with_recount{}_tabular.tex'.format(election.n))

In [85]:
# 1. Basic 20000/2000 plot without replacement

# data
election = Election(20000, 2000, 0.5, 1, False)

data_type = 'power.csv'
data_path = calibrated_table_dir/str(election)/data_type
data = prep_legend(read_csv(data_path))
filter_cols = filter(lambda x:x>=0.5, data.columns)
filter_cols = [col for i,col in enumerate(filter_cols) if i % 2 == 0]
data.index.name = 'Audit Method'
data[filter_cols].round(2).to_latex(report_tabular_dir/'power{}_tabular.tex'.format(election.n))

data_type = 'unconditional_mean.csv'
data_path = calibrated_table_dir/str(election)/data_type
data = prep_legend(read_csv(data_path))
filter_cols = filter(lambda x:x>=0.5, data.columns)
filter_cols = [col for i,col in enumerate(filter_cols) if i % 2 == 0]
data.index.name = 'Audit Method'
data[filter_cols].round(2).to_latex(report_tabular_dir/'unconditional_mean{}_tabular.tex'.format(election.n))

data_type = 'unconditional_mean_with_recount.csv'
data_path = calibrated_table_dir/str(election)/data_type
data = prep_legend(read_csv(data_path))
filter_cols = filter(lambda x:x>=0.5, data.columns)
filter_cols = [col for i,col in enumerate(filter_cols) if i % 2 == 0]
data.index.name = 'Audit Method'
data[filter_cols].round(2).to_latex(report_tabular_dir/'unconditional_mean_with_recount{}_tabular.tex'.format(election.n))